In [1]:
from tweepy import Stream
from tweepy import OAuthHandler
from tweepy.streaming import StreamListener
import tweepy
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pymysql
pd.options.display.max_columns = 50
pd.options.display.max_rows= 50
pd.options.display.width= 120

In [ ]:
consumer_key = "eKzcY1aUYnT5ra8Gu5g1vgH2X"
consumer_secret = "1slRr0TT7zwynm9K5V8NNBwQ6luCCa88OacSO9hTjRHu4aQ2lo"
api_key = "eKzcY1aUYnT5ra8Gu5g1vgH2X"
api_secret = "1slRr0TT7zwynm9K5V8NNBwQ6luCCa88OacSO9hTjRHu4aQ2lo"
access_token = "3165627764-b2PWTRlZ4HzqVvROoNYSdeZcN6bhiTfKeJsHBpo"
access_token_secret = "igsoovvvxde1jMsBeqNhdAugYzkG9ZkKy4dlwEpAUAZ6O"
auth = tweepy.OAuthHandler(api_key, api_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth)

In [ ]:
#streaming tweets for a keyword
print("Enter the keyword for streaming ...for example- 'Sundar Pichai' ")
Q = input()
results = api.search(q=Q)

Enter the keyword for streaming ...for example- 'Sundar Pichai' 


In [ ]:
tweet=results[0]

for param in dir(tweet):
    if not param.startswith("_"):
        print ("%s : %s" % (param, eval("tweet." + param)) )

In [ ]:
user=tweet.author

for param in dir(user):
    if not param.startswith("_"):
        print ("%s : %s" % (param, eval("user." + param)))

In [ ]:
results = []
for tweet in tweepy.Cursor(api.search, q=Q).items(500):
    results.append(tweet)

In [ ]:
def process_results(results):
    id_list = [tweet.text for tweet in results]
    data_set = pd.DataFrame(id_list, columns=["id"])

    # Processing Tweet Data

    data_set["text"] = [tweet.text for tweet in results]
    data_set["created_at"] = [tweet.created_at for tweet in results]
    data_set["retweet_count"] = [tweet.retweet_count for tweet in results]
    data_set["favorite_count"] = [tweet.favorite_count for tweet in results]
    data_set["source"] = [tweet.source for tweet in results]
    data_set["lang"] = [tweet.lang for tweet in results]

    # Processing User Data
    data_set["user_id"] = [tweet.author.id for tweet in results]
    data_set["user_screen_name"] = [tweet.author.screen_name for tweet in results]
    data_set["user_name"] = [tweet.author.name for tweet in results]
    #data_set["user_created_at"] = [tweet.author.created_at for tweet in results]
    #data_set["user_description"] = [tweet.author.description for tweet in results]
    data_set["user_followers_count"] = [tweet.author.followers_count for tweet in results]
    data_set["user_friends_count"] = [tweet.author.friends_count for tweet in results]
    data_set["user_location"] = [tweet.author.location for tweet in results]

    return data_set
data_set = process_results(results)
#set(data_set["lang"])

In [ ]:
data_set.to_csv('main_output.csv')
data_set.head()

In [ ]:
sources = data_set["source"].value_counts()[:5][::-1]

plt.barh(range(len(sources)), sources.values)
plt.yticks(np.arange(len(sources)) + 0.4, sources.index)
plt.show()

In [ ]:
#applying filters
print("Enter user name ELSE leave it blank ----- for example- 'Tech4SDGs' ")
u_name = input()
print('\n')

print("Enter user screen name ELSE leave it blank ----- for example- 'Tech4SDGs' ")
u_screen_name = input()
print('\n')

print("Enter Retweet Count (Greater or Equal to) ELSE leave it blank ----- for example- 5 ")
rt_count = input()
if(rt_count==''):
    rt_count=0
print('\n')

print("Enter Favourite Count (Greater or Equal to) ELSE leave it blank ----- for example- 5 ")
fav_count = input()
if(fav_count==''):
    fav_count=0
print('\n')

print("Enter Language ELSE leave it blank ----- for example- en for english, hi for hindi ")
language = input()
print('\n')

print("Enter User Location ELSE leave it blank ----- for example- 'India' ")
u_location = input()
print('\n')


#u_name = ""                    #Enter user_name (optional) e.g. u_name = "Tech4SDGs"
#u_screen_name = ""             #enter user_screen_name (optional) e.g. u_screen_name = "Tech4SDGs"
#rt_count = 0                   #enter retweet count
#fav_count = 0                  #enter favourite count
#language = ""                  #enter language e.g. language = "en"
#u_location = ""                #enter user's location e.g. u_location = "Global"
df = data_set
if u_name!='':
    df = df.loc[(data_set.user_name==u_name)]
if u_screen_name!='':
    df = df.loc[(data_set.user_screen_name==u_screen_name)]
df = df.loc[(data_set.retweet_count>=int(rt_count)) & (data_set.favorite_count>=int(fav_count))]
if u_location!='':
    df = df.loc[(data_set.user_location==u_location)]
if language!='':
    df = df.loc[(data_set.lang==language)]
df

In [ ]:
print("Enter the csv filename in which you want to export the filtered data ....for example: outputFile")
filename = input()
if(filename==''):
    filename='output.csv'
if((filename.split('.')[-1]) == 'csv'):
    filename = filename
else:
    filename = filename+'.csv'
df.to_csv(filename)

In [ ]:
sources = df["source"].value_counts()[:5][::-1]

plt.barh(range(len(sources)), sources.values)
plt.yticks(np.arange(len(sources)) + 0.4, sources.index)
plt.show()